In [ ]:
pip install faker

In [3]:
pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.7 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime

fake = Faker()

# Lista de produtos específicos com preço base
produtos = [
    (10, "Mouse", "Periféricos", 100),
    (11, "Teclado", "Periféricos", 150),
    (20, "Monitor", "Monitores", 800),
    (23, "Cabo HDMI", "Periféricos", 40),
    (30, "Notebook", "Computadores", 3500),
    (35, "Computador", "Computadores", 2300),
    (50, "Mesa", "Móveis", 450),
    (60, "Cadeira", "Móveis", 300),
    (70, "Licença Windows", "Software", 900),
    (80, "Licença Office 365", "Software", 350),
    (90, "HD SSD", "Armazenamento", 600),
    (95, "Memória", "Armazenamento", 320),
    (96, "Placa de Vídeo", "Periféricos", 2200)
]

garantiaExtendida = ['Sim', 'Não']

# Estruturas para armazenar cidades e vendedores fixos por país
estrutura_paises = {
    'Brasil': {
        'cidades': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte'],
        'vendedores': ['João', 'Ana', 'Carlos']
    },
    'Estados Unidos': {
        'cidades': ['New York', 'Los Angeles', 'Chicago'],
        'vendedores': ['John', 'Alice', 'Bob']
    },
    'Portugal': {
        'cidades': ['Lisboa', 'Porto', 'Coimbra'],
        'vendedores': ['Miguel', 'Sofia', 'André']
    },
    'Espanha': {
        'cidades': ['Madrid', 'Barcelona', 'Valencia'],
        'vendedores': ['Carlos', 'María', 'David']
    }
}

# Função para calcular o preço variável por mês
def calcular_preco_variavel(preco_base, data_venda):
    mes_venda = data_venda.month
    variacao_percentual = (mes_venda - 1) * 0.02  # Ajuste para 2% de variação por mês
    preco_variavel = round(preco_base * (1 + variacao_percentual), 2)
    return preco_variavel

# Número de registros desejados
num_linhas = 2000
num_duplicatas = 50

dados = []
for i in range(num_linhas):
    id_venda = i + 1  # Adicionando ID da venda
    produto = random.choice(produtos)
    garantia_extendida = random.choice(garantiaExtendida)
    data_venda = fake.date_between(start_date='-1y', end_date='today')
    codigo_produto, nome_produto, categoria_produto, preco_base = produto
    quantidade_vendido = random.randint(1, 10)
    valor_unitario = calcular_preco_variavel(preco_base, data_venda)
    valor_total_venda = round(quantidade_vendido * valor_unitario, 2)
    pais_loja = random.choice(list(estrutura_paises.keys()))
    cidade_loja = random.choice(estrutura_paises[pais_loja]['cidades'])
    vendedor = random.choice(estrutura_paises[pais_loja]['vendedores'])
    
    linha = [id_venda, data_venda, codigo_produto, nome_produto, garantia_extendida, categoria_produto, quantidade_vendido, valor_unitario, valor_total_venda, cidade_loja, pais_loja, vendedor]
    dados.append(linha)

# Adicionando algumas linhas duplicadas
for _ in range(num_duplicatas):
    dados.append(random.choice(dados))

# Criando o DataFrame
df = pd.DataFrame(dados, columns=['ID Venda', 'Data da Venda', 'Código Produto', 'Nome Produto', 'Garantia Extendida', 'Categoria do Produto', 'Quantidade Vendido', 'Valor Unitário', 'Valor Total da Venda', 'Cidade da Loja', 'País da Loja', 'Vendedor'])

# Salvando o DataFrame em um arquivo CSV
df.to_csv('dataset_variacao_preco_loja_informatica2.csv', index=False)

# Gerando dados dos clientes usando o ID da venda
dados_clientes = []
for linha in dados:
    id_venda = linha[0]  # Usando o ID da venda
    codigo_produto = linha[2]
    nome_cliente = fake.first_name()
    sobrenome_cliente = fake.last_name()
    data_compra = linha[1]  # Utilizando a data da venda como data da compra
    
    dados_clientes.append([id_venda, codigo_produto, nome_cliente, sobrenome_cliente, data_compra])

# Criando o DataFrame para os dados dos clientes
df_clientes = pd.DataFrame(dados_clientes, columns=['ID Venda', 'Codigo_Produto', 'Nome_Cliente', 'Sobrenome_Cliente', 'Data_Compra'])

# Salvando os DataFrames em arquivos Excel
with pd.ExcelWriter('vendas_informatica.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Vendas Produtos', index=False)
    df_clientes.to_excel(writer, sheet_name='Vendas por Cliente', index=False)